# Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, balanced_accuracy_score
warnings.filterwarnings("ignore")
from sklearn.model_selection import cross_val_score 
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval 
from hyperopt.early_stop import no_progress_loss
import pickle
import pandas as pd
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [3]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
# acc, balanced_acc, f1_score, binary_acc, ood_balanced_acc
def calculate_metrics(model, X,y):
    results = {}
    pred = model.predict(X)
    results["accuracy"] = accuracy_score(y, pred)
    results["balanced_accuracy"] = balanced_accuracy_score(y, pred)
    results["f1_micro"] = f1_score(y, pred, average="micro")


    y_pred_prob = model.predict_proba(X)[:, [0, 3, 2, 1]]
    surv = y_pred_prob[:,0]
    fail = np.sum(y_pred_prob[:,1:4], axis=1)
    y_pred_binary = (surv < fail).astype(int)
    
    y_binary = y.copy()
    y_binary[y_binary > 0] = 1


    
    results["binary_accuracy"] = accuracy_score(y_binary, y_pred_binary)
    #results["binary_balanced_accuracy"] = balanced_accuracy_score(y_binary, pred_binary)
    #results["binary_f1_micro"] = f1_score(y_binary, pred_binary, average="micro")
    #results["binary_f1_macro"] = f1_score(y_binary, pred_binary, average="macro")

    return results
    

#### Read the data

In [3]:
train_data = np.load("full_obs_data_train.npz")
val_data = np.load("full_obs_data_val.npz")
test_data = np.load("full_obs_data_test.npz")

dn = train_data["dn"]
senior = train_data["senior"]
topo = train_data["topo"]

dn = np.hstack([np.zeros((dn.shape[0],1)),dn])
senior = np.hstack([np.ones((senior.shape[0],1)),senior])
topo = np.hstack([2 * np.ones((topo.shape[0],1)),topo])

Xy_train = np.concatenate([dn,senior,topo],axis=0)
X_train, y_train = Xy_train[:, :-1], Xy_train[:,-1]


dn = val_data["dn"]
senior = val_data["senior"]
topo = val_data["topo"]

dn = np.hstack([np.zeros((dn.shape[0],1)),dn])
senior = np.hstack([np.ones((senior.shape[0],1)),senior])
topo = np.hstack([2 * np.ones((topo.shape[0],1)),topo])

Xy_val = np.concatenate([dn,senior,topo],axis=0)
X_val, y_val = Xy_val[:, :-1], Xy_val[:,-1]

dn = test_data["dn"]
senior = test_data["senior"]
topo = test_data["topo"]

dn = np.hstack([np.zeros((dn.shape[0],1)),dn])
senior = np.hstack([np.ones((senior.shape[0],1)),senior])
topo = np.hstack([2 * np.ones((topo.shape[0],1)),topo])

Xy_test = np.concatenate([dn,senior,topo],axis=0)
X_test, y_test = Xy_test[:, :-1], Xy_test[:,-1]

In [5]:
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical = True)
dval_clf = xgb.DMatrix(X_val, y_val, enable_categorical = True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical = True)

#dcv_clf = xgb.DMatrix(X_tr_val, y_tr_val, enable_categorical = True)

In [6]:
dn = np.load("OOD_DoNothing_obs_data.npy")
senior = np.load("OOD_Senior_original_95_obs_data.npy")
topo = np.load("OOD_Topo_Agent_95_2_obs_data.npy")

dn = np.hstack([np.zeros((dn.shape[0],1)),dn])
senior = np.hstack([np.ones((senior.shape[0],1)),senior])
topo = np.hstack([2 * np.ones((topo.shape[0],1)),topo])


Xy_test_ood = np.concatenate([dn, senior, topo])
X_test_ood, y_test_ood = Xy_test_ood[:,:-1], Xy_test_ood[:,-1]
print(X_test_ood.shape, y_test_ood.shape)
dtest_ood_clf = xgb.DMatrix(X_test_ood, y_test_ood, enable_categorical = True)


(6036, 4296) (6036,)


### XGBoost

### Train

In [7]:
clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2, objective="multi:softmax")


In [8]:
# Fit the model, test sets are used for early stopping.
clf.fit(X_train, y_train, eval_set=[(X_val, y_val)])
#with open('./models/xg_base_model.pkl', 'rb') as f:
#    clf = pickle.load(f)

[0]	validation_0-mlogloss:1.20525
[1]	validation_0-mlogloss:1.10395
[2]	validation_0-mlogloss:1.02724
[3]	validation_0-mlogloss:0.97411
[4]	validation_0-mlogloss:0.93480
[5]	validation_0-mlogloss:0.90148
[6]	validation_0-mlogloss:0.87567
[7]	validation_0-mlogloss:0.85342
[8]	validation_0-mlogloss:0.83495
[9]	validation_0-mlogloss:0.81629
[10]	validation_0-mlogloss:0.80208
[11]	validation_0-mlogloss:0.79094
[12]	validation_0-mlogloss:0.77621
[13]	validation_0-mlogloss:0.76621
[14]	validation_0-mlogloss:0.75853
[15]	validation_0-mlogloss:0.74931
[16]	validation_0-mlogloss:0.74006
[17]	validation_0-mlogloss:0.73152
[18]	validation_0-mlogloss:0.72376
[19]	validation_0-mlogloss:0.71754
[20]	validation_0-mlogloss:0.71111
[21]	validation_0-mlogloss:0.70570
[22]	validation_0-mlogloss:0.70161
[23]	validation_0-mlogloss:0.69628
[24]	validation_0-mlogloss:0.69047
[25]	validation_0-mlogloss:0.68534
[26]	validation_0-mlogloss:0.68099
[27]	validation_0-mlogloss:0.67514
[28]	validation_0-mlogloss:0.6

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=2,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [11]:
test_results = calculate_metrics(clf, X_test, y_test)
ood_results = calculate_metrics(clf, X_test_ood, y_test_ood) 
test_results["ood_balanced_accuracy"] = ood_results["balanced_accuracy"]
test_results["ood_binary_accuracy"] = ood_results["binary_accuracy"]

df_xg = pd.DataFrame([test_results])
#df.columns = ["train", "val", "test", "ood"]
df_xg.index = ["xg"]
df_xg

,accuracy,balanced_accuracy,f1_micro,binary_accuracy,ood_balanced_accuracy,ood_binary_accuracy
xg,0.803122,0.732749,0.803122,0.854564,0.730174,0.850729


In [12]:
params= {
    'tree_method':"hist", 
    'early_stopping_rounds':5,
    'objective':"multi:softprob",
    'num_class': 4,      
    'lambda': 5.912458896929504e-07, 
    'alpha': 0.19661638317561697, 
    'subsample': 0.8845438820749514, 
    'colsample_bytree': 0.9230103393659518, 
    'max_depth': 15, 
    'min_child_weight': 9, 
    'eta': 0.6926264634275339, 
    'gamma': 0.005152746565416908, 
    'grow_policy': 'lossguide'}

bst = xgb.train(params,dtrain_clf) 


In [15]:
preds = bst.predict(dtest_clf)
pred_labels = preds.argmax(axis=1)
accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
accuracy


0.7972137901127494

In [20]:
preds.argmax(axis=1)

array([0, 0, 0, ..., 2, 1, 1])

In [14]:
with open('./models/xg_best_new.pkl','wb') as f:
    pickle.dump(bst,f)

In [21]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
# acc, balanced_acc, f1_score, binary_acc, ood_balanced_acc
def calculate_metrics_xg(model, X,y):
    results = {}
    pred_probs = model.predict(X)
    pred = pred_probs.argmax(axis=1)

    results["accuracy"] = accuracy_score(y, pred)
    results["balanced_accuracy"] = balanced_accuracy_score(y, pred)
    results["f1_micro"] = f1_score(y, pred, average="micro")
    #results["f1_macro"] = f1_score(y, pred, average="macro")

    

    # binary 
    y_binary = y.copy()
    pred_binary = pred.copy()
    y_binary[y_binary > 0] = 1
    pred_binary[pred_binary > 0] = 1

      
    results["binary_accuracy"] = accuracy_score(y_binary, pred_binary)
    #results["binary_balanced_accuracy"] = balanced_accuracy_score(y_binary, pred_binary)
    #results["binary_f1_micro"] = f1_score(y_binary, pred_binary, average="micro")
    #results["binary_f1_macro"] = f1_score(y_binary, pred_binary, average="macro")

    return results
    

In [23]:
train_results = calculate_metrics_xg(bst, dtrain_clf, y_train)
val_results = calculate_metrics_xg(bst, dval_clf, y_val)
test_results = calculate_metrics_xg(bst, dtest_clf, y_test)
ood_results = calculate_metrics_xg(bst, dtest_ood_clf, y_test_ood) 

In [24]:
test_results["ood_balanced_accuracy"] = ood_results["balanced_accuracy"]
test_results["ood_binary_accuracy"] = ood_results["binary_accuracy"]

df_xg = pd.DataFrame([test_results])
#df_xg.columns = ["train", "val", "test", "ood"]
df_xg.index = ["XGBoost"]
df_xg

,accuracy,balanced_accuracy,f1_micro,binary_accuracy,ood_balanced_accuracy,ood_binary_accuracy
XGBoost,0.797214,0.730502,0.797214,0.833641,0.72784,0.831345
